In [1]:
'''
Retorna o maior valor do L
'''

def valor_maximo(c1, c2, lado, cima, diagonal):

    # retorna o valor da diagonal superior esquerda + 1 
    if (c1 == c2) and ((diagonal+1) > lado) and ((diagonal+1) > cima):
        diagonal += 1
        return diagonal

    # retorna o valor do lado esquerdo
    elif (lado > cima) and (lado > diagonal):
        return lado
  
    # retorna o valor de cima
    else:
        return cima


'''
Retorna o ponteiro do maior valor do L
'''

def acha_caminho(c1, c2, lado, cima, diagonal):
    # indica que o valor veio da diagonal esquerda superior
    if (c1 == c2) and ((diagonal+1) > lado) and ((diagonal+1) > cima):
        return "\\"

    # indica que o valor veio do lado esquerdo
    elif (lado > cima) and (lado > diagonal):
        return "-"
        
    # indica que o valor veio de cima
    else:
        return "|"


In [2]:
'''
Montar as matrizes:
- pontuacao (recebe os valores dos matches)
- caminho (recebe o caminho percorrido na matriz de acordo com o maior valor)

- seq1 fica na vertical, seq2 na horizontal
'''

def lcs(seq1, seq2):
    pontuacao = []
    caminho = []


    # preencher as linhas da matriz
    for i in range(0, len(seq1)):
        pontuacao.append([0] * len(seq2))
        caminho.append([""] * len(seq2))

    # preencher a primeira coluna com "|" e a primeira linha com "-" na matriz caminho
    for i in range(0, len(seq1)):
        caminho[i][0] = "|"
    for j in range(0, len(seq2)):
        caminho[0][j] = "-"

    # percorre as linhas
    for i in range(1, len(seq1)):

        # percorre as colunas
        for j in range(1, len(seq2)):

        # preenche a matriz com o maior valor do L (lado, cima, diagonal)
            pontuacao[i][j] = valor_maximo(seq2[j], seq1[i], pontuacao[i][j-1], pontuacao[i-1][j], pontuacao[i-1][j-1])

        # preenche a matriz com a indicação de onde veio o maior valor do L (lado, cima, diagonal)
            caminho[i][j] = acha_caminho(seq2[j], seq1[i], pontuacao[i][j-1], pontuacao[i-1][j], pontuacao[i-1][j-1])
        
    return pontuacao, caminho

In [3]:
'''
Imprimir a matriz
'''

def imprime_matriz(seq1, seq2, pontuacao, caminho):
    print("\t", end="")

    # imprime os caracteres da sequencia 2 na linha superior (horizontal) 
    for j in range(0, len(seq2)):
        print(seq2[j], end="\t")

    print()

    # imprime os caracteres da sequencia 1 na primeira coluna (vertical) e vai preenchendo as linhas com os valores
    for i in range(0, len(seq1)):
        print(seq1[i], end="\t")
        for j in range(0, len(seq2)):
            print(pontuacao[i][j], caminho[i][j], end="\t", sep="")

        print()
    print()


In [4]:
'''
Alinha as sequências
'''

def gera_alinhamento(seq1, seq2, pontuacao, caminho):
    ali_seq1 = ""
    ali_seq2 = ""

    # linhas
    i = len(seq1)-1
    # colunas
    j = len(seq2)-1

    while (i != 0) or (j != 0):
        # \\ indica que houve um match (imprime o mesmo caracter nas duas seqências)
        if caminho[i][j] == "\\":
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            i -= 1
            j -= 1

        # indical uma deleção na seq1 (imprime o caracter da seq2)
        elif caminho[i][j] == "-":
            ali_seq1 = "-" + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            j -= 1
            
        # indical uma deleção na seq2 (imprime o caracter da seq1)
        else:
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = "-" + ali_seq2
            i -= 1

    # imprime a pontuação (total de matches)
    print("Matches =", pontuacao[-1][-1])
    # imprime o alinhamento
    print(ali_seq2)
    print(ali_seq1)      

In [5]:
seq1 = "AAATTTACGTGGTT"
seq2 = "ACGTGT"


In [6]:
matriz_pontuacao, matriz_caminho = lcs(seq1, seq2)
imprime_matriz(seq1, seq2, matriz_pontuacao, matriz_caminho)
gera_alinhamento(seq1, seq2, matriz_pontuacao, matriz_caminho)

	A	C	G	T	G	T	
A	0-	0-	0-	0-	0-	0-	
A	0|	0|	0|	0|	0|	0|	
A	0|	0|	0|	0|	0|	0|	
T	0|	0|	0|	1\	1-	1-	
T	0|	0|	0|	1|	1|	2\	
T	0|	0|	0|	1|	1|	2|	
A	0|	0|	0|	1|	1|	2|	
C	0|	1\	1-	1|	1|	2|	
G	0|	1|	2\	2-	2-	2|	
T	0|	1|	2|	3\	3-	3-	
G	0|	1|	2|	3|	4\	4-	
G	0|	1|	2|	3|	4|	4|	
T	0|	1|	2|	3|	4|	5\	
T	0|	1|	2|	3|	4|	5|	

Matches = 5
------CGTG-T-
AATTTACGTGGTT
